In [ ]:
MAC_list={
# UNSW2018\2019
"d0:52:a8:00:67:5e":"Smart-Things",
"44:65:0d:56:cc:d3":"Amazon-Echo",
"70:ee:50:18:34:43":"Netatmo-Welcome",
"f4:f2:6d:93:51:f1":"TP-Link-Day-Night-Cloud-camera",
"00:16:6c:ab:6b:88":"Samsung-SmartCam",
"30:8c:fb:2f:e4:b2":"Dropcam",
"00:62:6e:51:27:2e":"Insteon-Camera",
"e8:ab:fa:19:de:4f":"unknown-maybe-cam",
"00:24:e4:11:18:a8":"Withings-Smart-Baby-Monitor",
"ec:1a:59:79:f4:89":"Belkin-Wemo-switch",
"50:c7:bf:00:56:39":"TP-Link-Smart-plug",
"74:c6:3b:29:d7:1d":"iHome",
"ec:1a:59:83:28:11":"Belkin-wemo-motion-sensor",
"18:b4:30:25:be:e4":"NEST-Protect-smoke-alarm",
"70:ee:50:03:b8:ac":"Netatmo-weather-station",
"00:24:e4:1b:6f:96":"Withings-Smart-scale",
"74:6a:89:00:2e:25":"Blipcare-Blood-Pressure-meter",
"00:24:e4:20:28:c6":"Withings-Aura-smart-sleep-sensor",
"d0:73:d5:01:83:08":"Light-Bulbs-LiFX-Smart-Bulb",
"18:b7:9e:02:20:44":"Triby-Speaker",
"e0:76:d0:33:bb:85":"PIX-STAR-Photo-frame",
"70:5a:0f:e4:9b:c0":"HP-Printer",
"08:21:ef:3b:fc:e3":"Samsung-Galaxy-Tab",
"30:8c:fb:b6:ea:45":"Nest-Dropcam",
"40:f3:08:ff:1e:da":"Android-Phone",
"74:2f:68:81:69:42":"Laptop",
"ac:bc:32:d4:6f:2f":"MacBook",
"b4:ce:f6:a7:a3:c2":"Android-Phone",
"d0:a6:37:df:a1:e1":"IPhone",
"f4:5c:89:93:cc:85":"MacBook/Iphone",
"14:cc:20:51:33:ea":"TPLink-Router-Bridge-LAN-(Gateway)",
#CIC
'1c:fe:2b:98:16:dd':'Amazon Alexa Echo Dot 1',
'a0:d0:dc:c4:08:ff':'Amazon Alexa Echo Dot 2',
'1c:12:b0:9b:0c:ec':'Amazon Alexa Echo Spot',
'08:7c:39:ce:6e:2a':'Amazon Alexa Echo Studio',
'cc:f4:11:9c:d0:00':'Google Nest Mini',
'48:a6:b8:f9:1b:88':'Sonos One Speaker',
'9c:8e:cd:1d:ab:9f':'AMCREST WiFi Camera',
'3c:37:86:6f:b9:51':'Arlo Base Station',
'40:5d:82:35:14:c8':'Arlo Q Camera',
'c0:e7:bf:0a:79:d1':'Borun/Sichuan-AI Camera',
'b0:c5:54:59:2e:99':'DCS8000LHA1 D-Link Mini Camera',
'44:01:bb:ec:10:4a':'HeimVision Smart WiFi Camera',
'34:75:63:73:f3:36':'Home Eye Camera',
'7c:a7:b0:cd:18:32':'Luohe Cam Dog',
'44:bb:3b:00:39:07':'Nest Indoor Camera',
'70:ee:50:68:0e:32':'Netatmo Camera',
'10:2c:6b:1b:43:be':'SIMCAM 1S (AMPAKTec)',
'b8:5f:98:d0:76:e6':'Amazon Plug',
'68:57:2d:56:ac:47':'Atomi Coffee Maker',
'8c:85:80:6c:b6:47':'Eufy HomeBase 2',
'50:02:91:b1:68:0c':'Globe Lamp ESP_B1680C',
'b8:f0:09:03:9a:af':'Gosund ESP_039AAF Socket',
'b8:f0:09:03:29:79':'Gosund ESP_032979 Plug',
'50:02:91:10:09:8f':'Gosund ESP_10098F Socket',
'c4:dd:57:0c:39:94':'Gosund ESP_0C3994 Plug',
'50:02:91:1a:ce:e1':'Gosund ESP_1ACEE1  Socket',
'24:a1:60:14:7f:f9':'Gosund ESP_147FF9 Plug',
'50:02:91:10:ac:d8':'Gosund ESP_10ACD8 Plug',
'd4:a6:51:30:64:b7':'HeimVision SmartLife Radio/Lamp',
'00:17:88:60:d6:4f':'Philips Hue Bridge',
'b0:09:da:3e:82:6c':'Ring Base Station AC:1236',
'50:14:79:37:80:18':'iRobot Roomba',
'00:02:75:f6:e3:cb':'Smart Board',
'd4:a6:51:76:06:64':'Teckin Plug 1',
'd4:a6:51:78:97:4e':'Teckin Plug 2',
'd4:a6:51:20:91:d1':'Yutron Plug 1',
'd4:a6:51:21:6c:29':'Yutron Plug 2',
'f0:b4:d2:f9:60:95':'D-Link DCHS-161 Water Sensor',
'ac:f1:08:4e:00:82':'LG Smart TV',
'70:ee:50:6b:a8:1a':'Netatmo Weather Station'}

# 1. Split raw traffic into sessions

In [ ]:
from scapy.all import rdpcap, wrpcap, IP, TCP, UDP, Ether
from collections import defaultdict
import os

def split_sessions_scapy(pcap_file, output_folder):
    packets = rdpcap(pcap_file)
    sessions = defaultdict(list)
    session_counter = defaultdict(int)

    for packet in packets:
        if IP in packet:
            src_ip = packet[IP].src
            dst_ip = packet[IP].dst
            proto = packet[IP].proto
            if TCP in packet or UDP in packet:
                src_port = packet.sport
                dst_port = packet.dport
                session_key = (src_ip, dst_ip, src_port, dst_port, proto)
            else:
                session_key = (src_ip, dst_ip, proto)
            sessions[session_key].append(packet)

    for session_key, pkts in sessions.items():
        src_ip = session_key[0]
        source_mac = pkts[0][Ether].src
        session_counter[src_ip] += 1
        session_number = session_counter[src_ip]

        label = MAC_list.get(source_mac, 'unknown')  

        output_dir = os.path.join(output_folder, label)
        os.makedirs(output_dir, exist_ok=True)

        output_filename = os.path.join(output_dir, f"{src_ip}_{session_number}.pcap")

        wrpcap(output_filename, pkts) 
        #print(f"Generated {output_filename} with {len(pkts)} packets.")

def process_all_pcaps(input_folder, output_folder):
    for pcap_file in os.listdir(input_folder):
        if pcap_file.endswith('.pcap'):
            pcap_path = os.path.join(input_folder, pcap_file)
            split_sessions_scapy(pcap_path, output_folder)

path_in = './UNSW2019/1pcap_all'
path_out = './UNSW2019/2session_pcap'

process_all_pcaps(path_in, path_out)


# 2. Normalize session files and eliminate mac and ip addresses

In [ ]:
from scapy.all import rdpcap, wrpcap, IP, Ether
import os
import shutil
import random
from pathlib import Path
# Set parameters equivalent to PowerShell variables
SESSIONS_COUNT_LIMIT_MIN = 0
SESSIONS_COUNT_LIMIT_MAX = 60000
TRIMED_FILE_LEN = 38756
SOURCE_SESSION_DIR = "UNSW2019/2session_pcap"

# Remove the source and destination MAC addresses and source and destination IP addresses from the packet
def remove_ip_mac(packets):
    for packet in packets:
        if IP in packet:
            # Clear the source and destination IP addresses
            packet[IP].src = "0.0.0.0"
            packet[IP].dst = "0.0.0.0"
        
        if Ether in packet:
            # Clear the source and destination MAC addresses
            packet[Ether].src = "00:00:00:00:00:00"
            packet[Ether].dst = "00:00:00:00:00:00"

    return packets


print(f"If Sessions more than {SESSIONS_COUNT_LIMIT_MAX}, we only select the largest {SESSIONS_COUNT_LIMIT_MAX}.")
print("Finally Selected Sessions:")

# Go through each directory in the source session directory
for d in Path(SOURCE_SESSION_DIR).iterdir():
    if d.is_dir():
        files = list(d.glob('*'))
        count = len(files)
        if count > SESSIONS_COUNT_LIMIT_MIN:
            print(f"{d.name} {count}")
            if count > SESSIONS_COUNT_LIMIT_MAX:
                # Sort files by size and take the largest SESSIONS_COUNT_LIMIT_MAX files
                files.sort(key=lambda f: f.stat().st_size, reverse=True)
                files = files[:SESSIONS_COUNT_LIMIT_MAX]

            # Create output directory for the combined set
            path_set = Path(f"UNSW2019/3final_pcap/FilteredSession/{d.name.lower()}")
            path_set.mkdir(parents=True, exist_ok=True)

            # Copy all files to the set directory
            for file in files:
                shutil.copy(file, path_set)

print("All files will be trimmed to {} length and if it's even shorter we'll fill the end with 0x00...".format(TRIMED_FILE_LEN))

# Creating trimmed versions for the set of files
input_path = Path("UNSW2019/3final_pcap/FilteredSession")
output_path = Path("UNSW2019/3final_pcap/TrimedSession")

for d in input_path.iterdir():
    if d.is_dir():
        output_dir = output_path / d.name.lower()
        output_dir.mkdir(parents=True, exist_ok=True)
        for f in d.glob('*'):
            packets = rdpcap(str(f))  
            packets = remove_ip_mac(packets)  
            output_filename = output_dir / f.name
            trimmed_content = b''.join(bytes(packet) for packet in packets)
            if len(trimmed_content) > TRIMED_FILE_LEN:
                trimmed_content = trimmed_content[:TRIMED_FILE_LEN]
            elif len(trimmed_content) < TRIMED_FILE_LEN:
                trimmed_content += bytes([0] * (TRIMED_FILE_LEN - len(trimmed_content)))

            with open(output_filename, 'wb') as out_file:
                out_file.write(trimmed_content)

print("Processing completed.")


# 3. Convert to grayscale

In [ ]:
import numpy
from PIL import Image
import binascii
import errno    
import os

PNG_SIZE = 28  # Size for the png images

def getMatrixfrom_pcap(filename, width):
    with open(filename, 'rb') as f:
        content = f.read()

    hexst = binascii.hexlify(content)  
    fh = numpy.array([int(hexst[i:i+2], 16) for i in range(0, len(hexst), 2)]) # Convert hexadecimal data to integer array  
    desired_size = width * width 
    if len(fh) < desired_size:
        # If the data length is insufficient, fill it with 0
        fh = numpy.pad(fh, (0, desired_size - len(fh)), 'constant')
    else:
        # If there is too much data, the first 784 elements are intercepted
        fh = fh[:desired_size]
    
    fh = numpy.reshape(fh, (width, width))  
    fh = numpy.uint8(fh)  # Convert to uint8 type
    return fh

def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc:
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

# Convert pcap to PNG and save to corresponding category-named folder
paths = [['UNSW2019/3final_pcap/TrimedSession', 'UNSW2019/4image_final']]

for source_path, target_path in paths:
    for category_name in os.listdir(source_path):
        category_dir = os.path.join(target_path, category_name)
        mkdir_p(category_dir)
        for filename in os.listdir(os.path.join(source_path, category_name)):
            source_file = os.path.join(source_path, category_name, filename)
            target_image = Image.fromarray(getMatrixfrom_pcap(source_file, PNG_SIZE))
            target_filename = os.path.splitext(filename)[0] + '.png'
            target_file_path = os.path.join(category_dir, target_filename)
            target_image.save(target_file_path)

print("All images have been converted and saved.")
